In [37]:
# | default_exp llms.streaming_handler

In [38]:
# | export
import re
import json
from typing import Dict, Any, Optional, Iterator
from rich.console import Console
from rich.markdown import Markdown
from agentic.configs.loader import get_reasoning_config

In [39]:
# | export
RESET = "\033[0m"
# Get reasoning config
reasoning_config = get_reasoning_config()
show_thinking = reasoning_config.get('show_thinking', True)
thinking_color = reasoning_config.get('thinking_color', 'pink')

# Set color based on config with enhanced colors
color_codes = {
            'pink': "\033[38;2;200;100;120m",     # Darker pink
            'blue': "\033[38;2;70;130;180m",      # Steel blue
            'green': "\033[38;2;60;179;113m",     # Medium sea green
            'yellow': "\033[38;2;204;204;0m",     # Olive-like dark yellow
            'purple': "\033[38;2;147;112;219m",   # Medium purple
            'cyan': "\033[38;2;0;139;139m"        # Dark cyan
        }


color_code = color_codes.get(thinking_color.lower(), color_codes['green'])   
def show_thinking_header():
    """Display beautiful thinking header"""
    if show_thinking:
        print(f"\n{color_code}╭─────────────────────── 🤔 Thinking ───────────────────────╮{RESET}")

def show_thinking_footer():
    """Display beautiful thinking footer"""
    if show_thinking:
        print(f"{color_code}╰────────────────────────────────────────────────────────────╯{RESET}")

In [40]:
# | export
class StreamingHandler:
    """Handles streaming responses"""
    
    def __init__(self):
        self.console = Console()
    
    def handle_streaming_response(self, response: Iterator, console: Optional[Console] = None) -> Dict[str, Any]:
        """Handle streaming response"""
        if console is None:
            console = self.console
        
        
        full_content = ""
        tool_calls = []
        first_token = True
        in_think_tag = False
        think_buffer = ""
        markdown_buffer = ""
        markdown_line_buffer = ""
        think_started = False
        
        def show_thinking_content(content):
            nonlocal think_started
            if show_thinking:
                if not think_started:
                    show_thinking_header()
                    print(f"{color_code}│ ", end="", flush=True)
                    think_started = True
                
                # enhanced_content = render_math_in_thinking(content)
                print(f"{color_code}{content}{RESET}", end="", flush=True)
            return think_started
    
        def flush_markdown_line():
            """Flush buffered markdown line if it ends with a newline."""
            nonlocal markdown_line_buffer
            if markdown_line_buffer.endswith('\n'):
                try:
                    console.print(Markdown(markdown_line_buffer.strip()))
                except:
                    print(markdown_line_buffer.strip())
                markdown_line_buffer = ""
        try:
            for chunk in response:
                if chunk.choices and chunk.choices[0].delta:
                    delta = chunk.choices[0].delta
        
                    # Reasoning content (not added to final message)
                    if hasattr(delta, 'reasoning') and delta.reasoning:
                        think_started = show_thinking_content(delta.reasoning)
        
                    if hasattr(delta, 'content') and delta.content:
                        content = delta.content
        
                        if '<think>' in content:
                            in_think_tag = True
                            parts = content.split('<think>')
                            if parts[0]:
                                markdown_line_buffer += parts[0]
                                full_content += parts[0]
                            flush_markdown_line()
                            think_buffer = parts[1] if len(parts) > 1 else ""
                            if think_buffer:
                                think_started = show_thinking_content(think_buffer)
                            continue
        
                        if '</think>' in content:
                            in_think_tag = False
                            parts = content.split('</think>')
                            if parts[0]:
                                think_buffer += parts[0]
                                think_started = show_thinking_content(parts[0])
                            if think_started:
                                print(f"{color_code} │{RESET}")
                                show_thinking_footer()
                                think_started = False
                            think_buffer = ""
                            if len(parts) > 1:
                                markdown_line_buffer += parts[1]
                                full_content += parts[1]
                                flush_markdown_line()
                            continue
        
                        if in_think_tag:
                            think_buffer += content
                            think_started = show_thinking_content(content)
                        else:
                            markdown_line_buffer += content
                            full_content += content
                            flush_markdown_line()
        
                        if first_token:
                            first_token = False
        
                    # Tool calls
                    if hasattr(delta, 'tool_calls') and delta.tool_calls:
                        for tool_call_delta in delta.tool_calls:
                            if tool_call_delta.index is not None:
                                while len(tool_calls) <= tool_call_delta.index:
                                    tool_calls.append({
                                        "id": "",
                                        "type": "function",
                                        "function": {"name": "", "arguments": ""}
                                    })
        
                                current_tool_call = tool_calls[tool_call_delta.index]
                                if tool_call_delta.id:
                                    current_tool_call["id"] = tool_call_delta.id
                                if tool_call_delta.function:
                                    if tool_call_delta.function.name:
                                        current_tool_call["function"]["name"] = tool_call_delta.function.name
                                    if tool_call_delta.function.arguments:
                                        current_tool_call["function"]["arguments"] += tool_call_delta.function.arguments
        
            # Flush any remaining markdown
            if markdown_line_buffer.strip():
                try:
                    console.print(Markdown(markdown_line_buffer.strip()))
                except:
                    print(markdown_line_buffer.strip())
        
            return {"content": full_content,
                    "tool_calls": tool_calls, 
                    "finish_reason":chunk.choices[0].finish_reason,
                    "usage": getattr(response, 'usage', None),
                    "model": getattr(response, 'model', None)}
        except Exception as e:
            console.print(f"[red]Error processing response: {e}[/red]")
            return {"content": "", "tool_calls": [], "error": str(e)}
    

In [41]:
# Example usage 
from agentic.llms.client import LLMClient
llm_client = LLMClient(model="qwen3:14b",base_url="http://192.168.29.147:11500/v1",api_key="ollama")
response = llm_client.create_completion(
            messages=[{"role":"user","content":"Why is sky blue?"}]
        )
result = llm_client.handle_streaming_response(response)


╭─────────────────────── 🤔 Thinking ───────────────────────╮
│ 
Okay, so I need to figure out why the sky is blue. Hmm, I remember from school that it has something to do with light and the atmosphere. Let me think... I think it's about the way sunlight interacts with the air. But wait, what's the exact process?

Alright, sunlight is white, but it's made up of different colors, right? Each color has a different wavelength. Maybe blue has a shorter wavelength? I think when sunlight enters the Earth's atmosphere, it collides with molecules and particles in the air. These collisions might cause the light to scatter in different directions. But why blue specifically?

Oh, right! There's something called Rayleigh scattering. I think Rayleigh scattering is when light scatters more if its wavelength is shorter. Since blue light has a shorter wavelength compared to, say, red or yellow, it scatters more. That means blue light is spread out in all directions, which is why we see the sky as blue

The sky appears blue due to a phenomenon called Rayleigh scattering, which involves the interaction of sunlight    
with the Earth's atmosphere. Here's a detailed breakdown:

1. Composition of Sunlight

Sunlight is a mixture of all visible wavelengths of light, each corresponding to a different color (from violet to 
red). However, the distribution of these wavelengths is not uniform—blue and green wavelengths are more prevalent  
than violet or red.

2. Rayleigh Scattering

...

 • Pollution or dust can alter sky color (e.g., red/orange haze) by introducing larger particles that change       
   scattering patterns.

6. Absence of Atmosphere

On the Moon, the sky appears black because there is no atmosphere to scatter sunlight, leaving only the blackness  
of space visible.

Summary

The sky is blue because blue light (shorter wavelength) scatters more efficiently than other colors in Earth's     
atmosphere due to Rayleigh scattering. This effect, combined with human visual sensitivity and atmospheric         
absorption, results in the blue-dominated sky we observe.

In [34]:
result.keys()

dict_keys(['content', 'tool_calls', 'usage', 'model'])